# Задание 1 #

In [1]:
import numpy as np
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

# Устанавливаем seed для повторяемости результатов
np.random.seed(42)
# Максимальное количество слов (по частоте использования)
max_words = 10000
# Максимальная длина рецензии в словах
max_len = 200

# Загружаем данные
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_words)

# Заполняем или обрезаем рецензии
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

# Создаем сеть
model = Sequential()
# Слой для векторного представления слов
model.add(Embedding(max_words, 8, input_length= max_len))
# Слой долго-краткосрочной памяти
model.add(LSTM(32, recurrent_dropout=0.2))
# Полносвязный слой
model.add(Dense(1, activation="sigmoid"))

# Копмилируем модель
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Обучаем модель
model.fit(X_train, y_train, epochs=3, batch_size=128, validation_split=0.1, verbose=1)

# Проверяем качество обучения на тестовых данных
scores = model.evaluate(X_test, y_test, verbose=1)
print("Точность на тестовых данных: %.2f%%" % (scores[1] * 100))

Epoch 1/3
176/176 [==============================] - 42s 223ms/step - loss: 0.5634 - accuracy: 0.7004 - val_loss: 0.3697 - val_accuracy: 0.8520
Epoch 2/3
176/176 [==============================] - 41s 233ms/step - loss: 0.2978 - accuracy: 0.8828 - val_loss: 0.3189 - val_accuracy: 0.8644
Epoch 3/3
782/782 [==============================] - 23s 30ms/step - loss: 0.3022 - accuracy: 0.8710
Точность на тестовых данных: 87.10%


In [2]:
# Сохраняем сеть для последующего использования
print("\nСохраняем сеть")
model_json = model.to_json()
json_file = open('saved_networks/lstm_net.json', "w")
json_file.write(model_json)
json_file.close()
model.save_weights('saved_networks/lstm_net.h5')
print("Сохранение сети завершено")


Сохраняем сеть
Сохранение сети завершено


# Задание 2 #

In [3]:
import numpy as np
from keras.utils import pad_sequences
from keras.models import model_from_json
from keras.datasets import imdb

max_words = 10000
max_len = 200
# Список классов
classes = ['негативный', 'положительный']

json_file = open('saved_networks/lstm_net.json', "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('saved_networks/lstm_net.h5')

loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Переводим текст рецензии в нужный вид
word_to_id = imdb.get_word_index()
word_to_id = {k: (v + 3) for k, v in word_to_id.items()}
file = open('reviews/negative.txt', "r")
text = file.read()
text = text.replace(',', '').replace('.', '').replace('!', '').replace('?', '').replace(':', '').replace(';', '')
words = text.split()
review = []
words = [word.lower() for word in words]
seq = []
for word in words:
    if word in word_to_id and word_to_id[word] < max_words:
        seq.append(word_to_id[word])
    else:
        seq.append(2)
x = np.array([seq])
x = pad_sequences(x, maxlen=max_len)

prediction = loaded_model.predict(x)

print(prediction)
print(classes[round(prediction[0][0])])

1/1 [==============================] - 1s 903ms/step
[[0.05708293]]
негативный
